In [ ]:
#data set link : https://www.kaggle.com/datasets/elvinagammed/chatbots-intent-recognition-dataset

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from google.colab import drive
import json
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/supervisedMLKaggle')


In [ ]:
# Opening JSON file
f = open('Intent.json')

# returns JSON object as
# a dictionary
data = json.load(f)

In [ ]:

data

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import io
import nltk
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalAveragePooling1D,Flatten, Dropout , GRU
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Conv1D, MaxPool1D

Lire notre Dataset comme étant un fichier JSON.

In [ ]:
for key in data['intents']:
  print(key)

In [ ]:
data['intents'][0]

In [ ]:
type(data['intents'][0])

In [ ]:
data['intents'][0].get('intent')

In [ ]:
print(data['intents'][0])

In [ ]:
(data['intents'][0]).get('intent')

In [ ]:
def intent_texte(data):
  intents = []
  texts = []
  for ligne in data['intents']:
    intent= ligne.get('intent')
    if 'text' in ligne:
      for texte in ligne['text']:
          print(texte)
          texte=texte
          print('intent is ', intent)
          intent=intent
          intents.append(intent)
          texts.append(texte)
  return intents, texts
#s(data['intents'][0]).get('intent')

In [ ]:
h, f=intent_texte(data)
print(h, f)

In [ ]:
dataset1=pd.DataFrame({"inputs":f , "tags": h})

In [ ]:
dataset1

In [ ]:
dataset1.columns

In [ ]:
dataset1.info()

In [ ]:
dataset1['inputs']=dataset1['inputs'].apply(lambda x : x.lower())

In [ ]:
import string

In [ ]:
dataset1['inputs']

In [ ]:
#on convertit les caractères en miniscule seulement si le caractère n'est pas dans la chaine de ponctuation
dataset1['inputs']= dataset1['inputs'].apply(lambda x : [caractere for caractere in x if caractere not in string.punctuation] )

In [ ]:
dataset1

In [ ]:
#concaténation des caractères au niveau des lignes de notre dataset
dataset1['inputs']= dataset1['inputs'].apply(lambda mot : ''.join(mot))

In [ ]:
dataset1['inputs']

In [ ]:
tokenizer= Tokenizer(num_words=1000)
tokenizer.fit_on_texts(dataset1['inputs']) #mettre à jour le dictionnaire interne du Tokenizer.
train = tokenizer.texts_to_sequences(dataset1['inputs'])  #convertir le texte en séquences d'entiers en utilisant le dictionnaire interne du Tokenizer.
features = pad_sequences(train)
le=LabelEncoder() #convertir les données textuelles en données étiquetées.
labels=le.fit_transform(dataset1['tags']) #cette méthode de la classe Label Encoder prends les étiquettes de la colonne 'tags' et les transforme en entiers.

In [ ]:
features

In [ ]:
labels

In [ ]:
len(features[0])

In [ ]:
features.shape

In [ ]:
vocabulary = len(tokenizer.word_index)
print('number of unique words : ', vocabulary)

In [ ]:
output_length = le.classes_.shape[0]
print('output length : ', output_length)

In [ ]:
tokenizer.word_index #mappe les mots uniques dans notre corpus avec leurs index, ce qui permet de visualiser la correspondance entre le corpus et son index
#on a les mots uniques dans la clé et la valeur c'est l'index  du mot

BUILD RNN MODEL

In [ ]:
m = Sequential() #initialisation d'un modèle de réseau de neurones séquentiel
m.add(Input(shape=(features.shape[1]))) #on ajoute une couche d'entrée au modèle
m.add(Embedding(vocabulary + 1,100)) #cpuche d'embeding au modèle
m.add(Conv1D(filters=32, kernel_size=5, activation="relu", kernel_initializer=tf.keras.initializers.GlorotNormal(),bias_regularizer=tf.keras.regularizers.L2(0.0001), kernel_regularizer=tf.keras.regularizers.L2(0.0001), activity_regularizer = tf.keras.regularizers.L2(0.0001))) # Ajoute une couche de convolution 1D avec 32 filtres et une taille de noyau de 5. Cette couche est suivie d'une fonction d'activation ReLU et de régularisations pour le biais, les poids et l'activité.
m.add(Dropout(0.3)) #Ajoute une couche de dropout avec un taux de dropout de 0.3 pour régulariser le modèle et prévenir le surapprentissage.
m.add(LSTM(32, dropout=0.3,return_sequences=True)) #Ajoute une couche LSTM avec 32 unités, un taux de dropout de 0.3 et return_sequences=True, ce qui signifie que cette couche renvoie des séquences au lieu d'une seule sortie.
m.add(LSTM(16, dropout=0.3,return_sequences=False)) #
m.add(Dense(128,activation="relu", activity_regularizer = tf.keras.regularizers.L2(0.0001)))
m.add(Dropout(0.6))
m.add(Dense(output_length, activation="softmax", activity_regularizer = tf.keras.regularizers.L2(0.0001)))


On utilise ce modèle pour la classification de texte,où les séquences de texte sont représentés sous forme d'entiers.
Le modèle utilise des couches d'embedding, de convolution et de LSTM pour extraire des caractéristiques pertinentes avant de les passer à une couche de sortie softmax pour la classification.

In [ ]:
! wget https://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
m.layers

In [ ]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
replace glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
glove_dir = "glove.6B.100d.txt"
embeddings_index = {}
file_ = open(glove_dir)
for line in file_:
    arr = line.split()
    single_word = arr[0]
    w = np.asarray(arr[1:],dtype='float32')
    embeddings_index[single_word] = w
file_.close()
print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
max_words= vocabulary + 1
word_index = tokenizer.word_index
embedding_matrix = np.zeros((max_words, 100)).astype(object)
for word , i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i]= embedding_vector

In [ ]:
m.layers[0].set_weights([embedding_matrix])
m.layers[0].trainable = False


In [ ]:
m.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])


In [ ]:

m.summary()

In [ ]:
from keras.callbacks import TensorBoard, EarlyStopping
earlyStopping = EarlyStopping(monitor = 'loss', patience = 400, mode = 'min', restore_best_weights = True)

In [ ]:
history_training = m.fit(features,labels,epochs=2000, batch_size=64, callbacks=[ earlyStopping])


In [ ]:
import matplotlib as mpl
def draw_plot(data, type_data):
  mpl.style.use('seaborn')
  plt.figure(figsize = (25, 5))
  plt.plot(data,'darkorange', label='Train')
  plt.xlabel('Epoch')
  plt.ylabel(type_data)
  plt.legend()

In [ ]:
draw_plot(history_training.history['accuracy'],'training set accuracy' )


In [ ]:
draw_plot(history_training.history['loss'],'training set loss' )


In [ ]:
m.evaluate(features, labels, batch_size = 64)


Live Testing

In [ ]:
import random
def generate_answer(query):
  texts = []
  pred_input = query
  pred_input = [letters.lower() for letters in pred_input if letters not in string.punctuation]
  pred_input = ''.join(pred_input)
  texts.append(pred_input)
  pred_input = tokenizer.texts_to_sequences(texts)
  pred_input = np.array(pred_input).reshape(-1)
  pred_input = pad_sequences([pred_input],input_shape)
  output = m.predict(pred_input)
  output = output.argmax()
  response_tag = le.inverse_transform([output])[0]
  return random.choice(responses[response_tag])

In [ ]:
list_que = ["hello", "i am kaled","what is my name?",
            "what is your name?", "tell me please, what is your name?"]
for i in list_que:
  print("you: {}".format(i))
  res_tag = generate_answer(i)
  print(res_tag)